In [1]:
import os
import sys
from importlib import reload
from pathlib import Path

import numpy as np
from torch.nn import ReLU, Linear
from torch_geometric.nn import GCNConv

In [2]:
# NOTE: Boilerplate setup for Jupyter imports
root = Path(os.getcwd()).parent
sys.path.append(
    (root / "src").as_posix()
)

import configs as config_module
import enhancer as enhancer_module
import encoders as encoders_module
import scheme.edges as edges_module
import scheme.network as network_module
import scheme.data as data_module

reload(edges_module)
reload(config_module)
reload(enhancer_module)
reload(encoders_module)
reload(network_module)
reload(data_module)

PathConfig, TrainConfig = config_module.PathConfig, config_module.TrainConfig
Enhancer = enhancer_module.Enhancer
get_default_encoders = encoders_module.get_default_encoders
GNNConfig, EnhancerData = network_module.NetworkConfig, data_module.EnhancerData
EdgeStrategy = edges_module.EdgeStrategy

In [3]:
path_config = PathConfig(
    data_root=(root / "data").as_posix()
)

with open(path_config.data_root / "processed/np/cora.npz", "rb") as f:
    unpacked = np.load(f)

    # NOTE: Target dimensions
    data = EnhancerData(
        unpacked["data"],
        unpacked["target"].reshape(-1),
        unpacked["spatial"],
    )

In [4]:
gnn_setup = GNNConfig(
    encoder=[
        GCNConv(data.features.shape[1], 256),
        GCNConv(256, 256),
    ],
    estimator=[
        Linear(256, 128),
        ReLU(),
        Linear(128, 128),
        ReLU(),
        Linear(128, len(np.unique(data.target))),
    ]
)

In [5]:
encoders = [
    encoders_module.DistEncoder(
        max_dist=5,
        cache_dir=path_config.edge_cache,
        note="cora_dist",
    ),
    encoders_module.ReprEncoder(
        neighbor_rate=0.7,
        cache_dir=path_config.edge_cache,
        note="cora_repr",
    ),
]

strategies = [
    EdgeStrategy(e, data)
    for e in encoders
]

result = Enhancer.compare_builders(gnn_setup, strategies)

cora_dist test: Loss = 1.4488e+00 | Accuracy = 0.6752767527675276
cora_repr test: Loss = 1.2929e+00 | Accuracy = 0.5350553505535055


In [6]:
print(result)

Option       accuracy_score    f1_score
---------  ----------------  ----------
cora_dist          0.675277    0.642889
cora_repr          0.535055    0.527314


In [7]:
enhancer = Enhancer(
    net_config=gnn_setup,
    edge_builder=encoders_module.DistEncoder(
        max_dist=5,
        cache_dir=path_config.edge_cache,
        note="cora_dist [max dist = 5]",
    ),
)

In [8]:
gnn = enhancer.fit(data)

In [9]:
enhancer.transform(data)

array([[-0.14913446, -0.22291206, -0.2982183 , ..., -0.13839063,
         0.4110715 ,  0.41540155],
       [-0.5775375 , -0.74322426, -0.37276083, ..., -0.59690994,
         0.24487399, -0.30890217],
       [-0.25104582, -1.0337194 , -0.07432133, ..., -0.46774113,
         0.2804356 , -0.40491274],
       ...,
       [ 0.0550571 , -0.57310057, -0.7471699 , ...,  0.1554817 ,
         0.5920073 ,  0.24447809],
       [-0.19284065, -0.6017901 , -0.4805082 , ..., -0.28493458,
         0.2846866 ,  0.27027017],
       [-0.17198163, -0.53422344, -0.5038126 , ...,  0.08991784,
         0.44923875,  0.35963517]], dtype=float32)